This year's Halloween playground competition we are challenged to predict the author of excerpts from horror stories by **Edgar Allan Poe**, **Mary Shelley**, and **HP Lovecraft**. Wish you all a very happy HALLOWEEN
![Happy Halloween](https://78.media.tumblr.com/ef1008036eb43692b6888d0b68b2a72c/tumblr_oxmujdjP891wd3q54o1_500.gif)

### Import libraries

Let us start with importing all the important libraries and defining useful functions for this kernel. 

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import re
import string
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import gensim
import codecs
from scipy.misc import imread
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [ ]:
def clean_data(raw_doc):
    tokenized_doc = word_tokenize(raw_doc)
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    new_review = []
    for token in tokenized_doc: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token) 
    
    snowball = SnowballStemmer('english')
    final_doc = []
    for word in new_review:
        final_doc.append(snowball.stem(word))
    
    return final_doc

def generate_word_clouds(author, pallete):
    # The wordcloud of Cthulhu/squidy thing for HP Lovecraft
    plt.figure(figsize=(14,11))
    wc = WordCloud(background_color="black", max_words=350, 
                   mask=hcmask, max_font_size= 130,stopwords=set())
    wc.generate(" ".join(train_df[train_df["author"]==author]["text"]))
    plt.title(author, fontsize=16)
    plt.imshow(wc.recolor( colormap= pallete, random_state=17), alpha=0.98)

def top_frequent_words(author):    
    counts = Counter(clean_data(' '.join(train_df[train_df["author"]==author]["text"])))
    labels, values = zip(*counts.items())
    indSort = np.argsort(values)[-30:]

    # rearrange your data
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]

    indexes = np.arange(len(labels))
    bar_width = 0.35

    plt.figure(figsize=(12,8))
    sns.set(style="ticks")
    sns.set_palette(palette="hot")
    sns.barplot(labels[indexes], values)
    plt.ylabel('Word Count', fontsize=14)
    plt.xlabel('Word', fontsize=14)
    plt.title("Word Frequency distribution", fontsize=18)
    plt.show()
    
def top_tfidf_feats(row, features, top_n=10):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_feats_in_doc(Xtr, features, row_id, top_n=10):
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=10):
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=10):
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

def plot_tfidf_classfeats_h(dfs):
    fig = plt.figure(figsize=(12, 100), facecolor="w")
    x = np.arange(len(dfs[0]))
    for i, df in enumerate(dfs):
        #z = int(str(int(i/3)+1) + str((i%3)+1))
        ax = fig.add_subplot(9, 1, i+1)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.set_frame_on(False)        
        c = cm.hot((i+1)/10.,1)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        ax.set_xlabel("Mean Tf-Idf Score", labelpad=16, fontsize=16)
        ax.set_ylabel("Word", labelpad=16, fontsize=16)
        ax.set_title(str(df.label), fontsize=18)
        ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
        ax.barh(x, df.tfidf, align='center',color=c)
        ax.set_yticks(x)
        ax.set_ylim([-1, x[-1]+1])
        yticks = ax.set_yticklabels(df.feature)
        plt.subplots_adjust(bottom=0.09, right=0.97, left=0.15, top=0.95, wspace=0.52)
    plt.show()

### Data Summary

Quick look at the file sizes on input files to get an understanding on how big the data is

In [ ]:
for f in os.listdir('../input'):
    size_bytes = round(os.path.getsize('../input/' + f)/ 1000, 2)
    size_name = ["KB", "MB"]
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    print(f.ljust(20) + str(s).ljust(7) + size_name[i])

Data is quite small. Let's get started by loading this data to DataFrame and having a glance on them

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df['text_length'] = train_df['text'].str.len()

It should be a good idea to look at how the training data is distributed among the authors.

In [ ]:
plt.figure(figsize=(12,8))
sns.set(style="ticks")
sns.set_palette(palette="Blues")
sns.countplot(x="author", data=train_df)
plt.ylabel('Frequency', fontsize=14)
plt.xlabel('Author', fontsize=14)
plt.title("Author Frequency distribution", fontsize=18)
plt.show()

As we can see **EAP** is having **more text **in training than other two authors. It would also be good to have glance at how long does a paragraph written by author is.

In [ ]:
plt.figure(figsize=(12,8))
sns.set(style="ticks")
sns.set_palette(palette="BuGn_r")
sns.stripplot(x="author", y="text_length", data=train_df);
plt.ylabel('Text Count', fontsize=14)
plt.xlabel('Author', fontsize=14)
plt.title("Text length distribution", fontsize=18)
plt.show()

MWS likes to include more text in his paragraphs that other two authors. Even though EAP has more text than any author likes to keep it short . His text has rarely crossed 1000 word count mark.

### Word Cloud

I am a big fan of Sir Arthur Conan Doyle. So, I would like to plot a word cloud over Sherlock Holme's silhouette. 

Note: Unlike every other NLP problem, here we also need to know which stopwords are mostly used by an author. Because it may help us to know the writing pattern on an author.

In [ ]:
sherlock = b'...'

In [ ]:
# Generate the Mask for EAP
f1 = open("sherlock.png", "wb")
f1.write(codecs.decode(sherlock,'base64'))
f1.close()
img1 = imread("sherlock.png")
hcmask = img1

In [ ]:
palletes = ["hot", "Greys", "inferno"]
for author,pallete in zip(train_df["author"].unique(), palletes):
    generate_word_clouds(author, pallete)

It would also be helpful to have a look at the top occuring words in each authors text.

In [ ]:
for author in train_df["author"].unique():
    top_frequent_words(author)

### TF-IDF

Tf-idf stands for **term frequency-inverse document frequency**, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a** statistical measure** used **to evaluate how important a word is** to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.

Ref ~ http://www.tfidf.com/

Lets plot out some top words we got using Tf-Idf for each author

In [ ]:
tfidf = TfidfVectorizer(
    min_df=5, max_features=16000, strip_accents='unicode',lowercase =True,
    analyzer='word', token_pattern=r'\w+', use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words = 'english').fit(train_df["text"])

Xtr = tfidf.fit_transform(train_df["text"])
y = train_df["author"]
features = tfidf.get_feature_names()
top_dfs = top_feats_by_class(Xtr, y, features)
plot_tfidf_classfeats_h(top_dfs)

Words like say, said are important in case of Author EAP, which does not happen in case of Author HPL or MWS. Word Raymond is important in case of Author MWS.

### LDA and Topic Visualization

In [ ]:
import warnings
warnings.filterwarnings('ignore')
lda_tfidf = LatentDirichletAllocation(n_components=15, random_state=0, learning_method="batch")
lda_tfidf.fit(Xtr)
#pyLDAvis.sklearn.prepare(lda_tfidf, Xtr, tfidf, mds='mmds')

To be Continued...